In [85]:
import numpy as np
import pandas as pd
import random

Trained_data = pd.read_csv("basic_trained_data_20Percent.csv" , sep = "," , encoding = 'utf-8')
Test_data = pd.read_csv("basic_tested_data.csv" , sep = "," , encoding = 'utf-8')

#二進位轉十進位
def bin2dec(bin):
    result = 0
    for i in range(len(bin)):
        result += pow(2, i) * bin[i]
    return result

def generate_point(Qbits):
    point = []
    point.append((bin2dec(Qbits[0:4]) / 16) * Trained_data["duration"].max())
    point.append((bin2dec(Qbits[4:8]) / 16) * Trained_data["src_bytes"].max())
    point.append((bin2dec(Qbits[8:12]) / 16) * Trained_data["dst_bytes"].max())
    point.append(bin2dec(Qbits[12:14]))
    point.append(bin2dec(Qbits[14:16]))
    return point

def cal_distance(Point1, Point2):
    dis = 0
    for i in range(len(Point1)):
        dis += abs(Point1[i] - Point2[i])
    return dis

#fitness的計算根據問題會不一樣，這裡使用ABS當範例
def fitness(QTS, data):
    for i in range(QTS.population_size):
        QTS.fitness[i] = 0
        points = []
        points.append(generate_point(QTS.Population[i][0:16]))
        points.append(generate_point(QTS.Population[i][16:32]))
        for d in data:
            dis_safe = cal_distance(d[1:-1], points[0])
            dis_attack = cal_distance(d[1:-1], points[1])
            if (dis_safe < dis_attack and d[-1] == 0) or (dis_safe > dis_attack and d[-1] == 1):
                QTS.fitness[i] += 1
        QTS.fitness[i] /= len(data)
                       
class QTS:
    #初始化
    def __init__(self, population_size = 10, max_cycle = 100, Qbit_num = 10, theta = 0.01):
        self.population_size = population_size
        self.max_cycle = max_cycle
        self.Qbit_num = Qbit_num
        self.theta = theta
        self.Population = np.zeros((population_size, Qbit_num))
        self.Q_matrix = [0.5 for i in range(Qbit_num)]
        self.fitness = [None for i in range(population_size)]
        self.best_index = -1
        self.worst_index = -1
        self.best = [None for i in range(Qbit_num)]
    
    #設定旋轉角度
    def set_theta(self, new_theta):
        self.theta = new_theta
    
    #生成新的Population
    def generate_new_population(self):
        for i in range(self.population_size):
            for j in range(self.Qbit_num):
                self.Population[i][j] = int(self.Q_matrix[j] < random.random())
    
    #找出最佳及最差的粒子(min為最佳)
    def selection_find_min(self):
        self.best_index = self.fitness.index(min(self.fitness))
        self.worst_index = self.fitness.index(max(self.fitness))
        if self.best[-1] == None:
            self.best = self.Population[self.best_index]
            self.best = np.append(self.best, min(self.fitness))
        elif min(self.fitness) < self.best[-1]:
            self.best = self.Population[self.best_index]
            self.best = np.append(self.best, min(self.fitness))
    
    #找出最佳及最差的粒子(max為最佳)
    def selection_find_max(self):
        self.best_index = self.fitness.index(max(self.fitness))
        self.worst_index =self.fitness.index(min(self.fitness))
        if self.best[-1] == None:
            self.best = self.Population[self.best_index]
            self.best = np.append(self.best, max(self.fitness))
        elif max(self.fitness) > self.best[-1]:
            self.best = self.Population[self.best_index]
            self.best = np.append(self.best, max(self.fitness))
    
    #更新Qmatrix
    def update(self):
        for i in range(self.Qbit_num):
            if self.Population[self.best_index][i] != self.Population[self.worst_index][i]:
                if self.Population[self.best_index][i] == 1:
                    self.Q_matrix[i] -= self.theta
                else:
                    self.Q_matrix[i] += self.theta
                    
    #更新Qmatrix, Gbest
    def update_G(self):
        for i in range(self.Qbit_num):
            if self.best[i] != self.Population[self.worst_index][i]:
                if self.best[i] == 1:
                    self.Q_matrix[i] -= self.theta
                else:
                    self.Q_matrix[i] += self.theta
    
    #簡易測試
    def run(self):
        for i in range(self.max_cycle):
            self.generate_new_population()
            fitness(self)
            self.selection_find_min()
            self.update()
            print(self.fitness)
            print(self.best)

In [86]:
f = open("best_20Percent_Gbest.txt", "w")
random.seed(722)
data = np.array(Trained_data)
QTS_test = QTS(10, 10000, 32, 0.01)
for i in range(QTS_test.max_cycle):
    QTS_test.generate_new_population()
    fitness(QTS_test, data)
    QTS_test.selection_find_max()
    QTS_test.update_G()
    if i % 500 == 0:
        print("Now:", i)
        print(QTS_test.Q_matrix)
    for j in range(len(QTS_test.best)):
        if j != len(QTS_test.best)-1:
            print(str(int(QTS_test.best[j])), end=" ", file=f)
        else:
            print(str(QTS_test.best[j]), file=f)
print(QTS_test.fitness)
f.close()

Now: 0
[0.5, 0.49, 0.5, 0.49, 0.51, 0.51, 0.49, 0.51, 0.5, 0.49, 0.51, 0.51, 0.51, 0.49, 0.5, 0.5, 0.51, 0.5, 0.5, 0.51, 0.51, 0.51, 0.49, 0.5, 0.5, 0.5, 0.49, 0.49, 0.5, 0.51, 0.49, 0.5]
Now: 500
[-3.0878077872387166e-16, -3.0878077872387166e-16, 1.0000000000000004, 1.0000000000000004, 1.0000000000000004, 1.0000000000000004, -3.0878077872387166e-16, -3.0878077872387166e-16, 1.0000000000000004, -3.0878077872387166e-16, 1.0000000000000004, 1.0000000000000004, -3.0878077872387166e-16, -3.0878077872387166e-16, 0.009999999999999691, 1.0000000000000004, -3.0878077872387166e-16, 1.0000000000000004, 1.0000000000000004, -3.0878077872387166e-16, 1.0000000000000004, 1.0000000000000004, -3.0878077872387166e-16, -3.0878077872387166e-16, 1.0000000000000004, -3.0878077872387166e-16, 1.0000000000000004, 1.0000000000000004, 0.9700000000000004, 0.9900000000000004, -3.0878077872387166e-16, 0.9900000000000004]
Now: 1000
[-3.0878077872387166e-16, -3.0878077872387166e-16, 1.0000000000000004, 1.000000000000

KeyboardInterrupt: 

In [81]:
f.close()

In [ ]:
#範例
Trained_data = pd.read_csv("basic_trained_data_20Percent.csv" , sep = "," , encoding = 'utf-8')
Test_data = pd.read_csv("basic_tested_data.csv" , sep = "," , encoding = 'utf-8')
data = np.array(Trained_data)

random.seed(722)
QTS_1 = QTS(100, 10000, sum(encode_num), 0.004)
for i in range(QTS_1.max_cycle):
    QTS_1.generate_new_population()
    fitness(QTS_1, data)
    QTS_1.selection_find_max()
    QTS_1.update
print(QTS_1.best)

[0.5345791128187216,
 0.5345791128187216,
 0.5345791128187216,
 0.5345791128187216,
 0.5345791128187216,
 0.5345791128187216,
 0.5345791128187216,
 0.5345791128187216,
 0.5345791128187216,
 0.4529101705140825]

In [82]:
QTS_0.Q_matrix

[1.0000000000000004,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.163336342344337e-16,
 1.0000000000000004,
 1.0000000000000004,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 1.0000000000000004,
 1.0000000000000004,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.163336342344337e-16,
 -4.163336342344337e-16,
 1.0000000000000004,
 1.0000000000000004,
 -4.163336342344337e-16,
 1.0000000000000004,
 -4.1

In [46]:
Trained_data = pd.read_csv("basic_trained_data.csv" , sep = "," , encoding = 'utf-8')
Test_data = pd.read_csv("basic_tested_data.csv" , sep = "," , encoding = 'utf-8')
data = np.array(Trained_data)

array([1, 0, 0, 0, 0, 0, 1], dtype=int64)

In [78]:
f.close()

In [34]:
encode_num

[17, 32, 32, 3, 5]